In [85]:
import pandas as pd 
import numpy as np 
import re
import json 

What is the state breakdown of the gendered phrases?

In [100]:
masculine_themed_wording = [
    "competitive",
    "dominate",
    "leader",
    "rock star",
    "rockstar",
    "guru",
    "ninja",
    "hacker",
    "superhero",
    "prove themselves",
    "analyze",
    "determine",
    "crush it",
    "world class",
    "superior",
    "ambitious",
    "aggressive",
    "leader"
]

In [82]:
data_set = pd.read_csv("dice_com-job_us_sample.csv")
data_set = data_set.get(['jobtitle','jobdescription','joblocation_address'])
states = pd.read_csv("states.csv")

In [ ]:
data_set['State'] = data_set['joblocation_address'].apply(lambda x: pd.Series(str(x).split(", ")[-1]))

In [94]:
def numOfOcurrences(phrase, description): 
    regex = re.compile(phrase)
    return len(re.findall(regex, description))

In [114]:
def indicesOfJobsWithPhrase(phrase, df): 
    indices = []
    for index, row in df.iterrows(): 
        if numOfOcurrences(phrase, row.get("jobdescription")) > 0: 
            indices += [index]
    return indices

In [128]:
def computeStateInfo(state, full_df): 
    stateInfo = {}
    stateInfo['word_count'] = {}
    state_df = full_df.where(full_df['State'] == state).dropna()
    biasedIndices = []; 
    for word in masculine_themed_wording: 
        
        indices = indicesOfJobsWithPhrase(word, state_df)
        stateInfo['word_count'][word] = len(indices)
        biasedIndices += indices
    
    stateInfo['region'] = state
    stateInfo['numberOfJobsInDataSet'] = len(state_df)
    stateInfo['numberOfBiasedJobs'] = len(set(biasedIndices))
    stateInfo['biasedIndices'] = list(set(biasedIndices))
    return stateInfo
    

In [134]:
def exportInfo(DF): 
    data = [computeStateInfo(state, DF) for state in states['Code']]
    stateData = {"data": data}; 
    with open('stateData.json', 'w') as outfile:
        json.dump(stateData, outfile)
    
        

In [135]:
exportInfo(data_set)